In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy import types
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

import os
import sys
sys.path
sys.path.append('../')
from connections import password, client_id, client_secret

In [2]:
# Client credentials
client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id, 
    client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# Establish SQL connection
connection_string = (f"root:{password}@localhost/spot_db")
engine = create_engine(f"mysql://{connection_string}")# , pool_recycle=3600, pool_pre_ping=True)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# Save reference to the table
_songs = Base.classes.songs
try:
    features = Base.classes.features
except:
    pass
# Create connection object
session = Session(engine)

In [4]:
genres = sp.recommendation_genre_seeds()
genres = genres['genres']

In [5]:
import random
shuffled_genres = random.shuffle(genres)

In [17]:
def chunk_lists(list, n = 5): 
    for i in range(0, len(list), n):  
        yield list[i:i + n] 
genre_chunks = list(chunk_lists(genres))
genre_chunks[0:4]

[['pop-film', 'iranian', 'techno', 'punk', 'singer-songwriter'],
 ['punk-rock', 'ambient', 'german', 'piano', 'road-trip'],
 ['funk', 'guitar', 'holidays', 'bluegrass', 'indie'],
 ['hard-rock', 'spanish', 'brazil', 'british', 'romance']]

In [18]:
rec_tests = []
for chunk in genre_chunks:
    rec_tests.append(sp.recommendations(seed_genres = chunk, limit = 100))

In [35]:
len(unique_songs)

2548

In [21]:
ids = []
tracks = []
artists = []
for i in range(0, len(rec_tests)):
    for j in range(0, len(rec_tests[i]['tracks'])):
        ids.append(rec_tests[i]['tracks'][j]['id'])
        tracks.append(rec_tests[i]['tracks'][j]['name'])
        artists.append(rec_tests[i]['tracks'][j]['artists'][0]['name'])

In [30]:
songs_df = pd.DataFrame(data = {'ID':ids, 'Track':tracks, 'Artist':artists})

In [32]:
song_id = session.query(_songs.ID).all()
id_list = list(np.ravel(song_id))

In [34]:
unique_songs = songs_df.loc[~songs_df.ID.isin(id_list)]

In [36]:
unique_songs.drop_duplicates(inplace = True)

C:\Users\Will\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
for id in id_list:
    if id in unique_songs.ID:
        print(id)

In [38]:
engine.execute("USE spot_db")
unique_songs.to_sql(
    name = 'songs', con = engine,
    if_exists = 'append')
# with engine.connect() as con:
#     con.execute('ALTER TABLE `songs` ADD PRIMARY KEY (`ID`);')